# Calculate BPR coefficients for a street network

This notebook calculates the BPR (Bureau of Public Roads) curves' coefficients $a_0$, $a_1$, $a_2$, $a_3$, and $a_4$ for an OpenStreetMap street network of the bay area.

## BPR function definition:

$t_i = t0_i (1 + \alpha (\frac{v_i}{c_i}) ^ \beta)$

where:

$t_i =$ congested flow travel time on link $i$

$t0_i =$ free-flow travel time on link $i$

$v_i$ = volume of traffic on link $i$ per unit of time (flow attempting to use link $i$)

$c_i =$ capacity of link $i$ per unit of time

$\alpha =$ alpha coefficient, which was assigned a value of 0.15 in the original BPR curve

$\beta =$ beta coefficient, the exponent of the power function, which was assigned a value of 4 in the original BPR curve

We will use these default $\alpha$ and $\beta$ parameter values in this script, but note that they are not the only way to parameterize this function. For instance, TRB's *NHCRP Report 716: Travel Demand Forecasting Parameters and Techniques* pg 75 provides various coefficients estimated using the 1985 *Highway Capacity Manual*.


## BPR coefficients to be calculated in this script:

$a_0 = t0_i =$ the free-flow travel time

$a_1$, $a_2$, and $a_3$ are not used, they are always zero

$a_4 = t0_i \frac{\alpha}{c_i ^ {\beta}}$


## Our edge capacity and speed defaults, by highway type and number of lanes:

### Capacity defaults:

| OSM hwy type  | Description | lanes=1 | lanes=2 | lanes=3 | lanes=4+|
| ------------- | ------------- | ------------- | ------------- | ------------- | ------------- |
| motorway  | freeway, turnpike, or interstate  | 1900 | 2000 | 2000 | 2200 |
| trunk  | surface expressway, divided road, limited number of intersections  | 1900 | 2000 | 2000 | 2000 |
| primary  | arterial road  | 1000 | 1000 | 1000 | 1000 |
| secondary  | usually separated by a central line on road  | 900 | 900 | 900 | 900 |
| tertiary  | links local centers of activity with low to moderate traffic  | 900 | 900 | 900 | 900 |

### Max speed defaults:

| OSM hwy type  | Description | lanes=1 | lanes=2 | lanes=3 | lanes=4+|
| ------------- | ------------- | ------------- | ------------- | ------------- | ------------- |
| motorway  | freeway, turnpike, or interstate  | 50 | 50 | 65 | 65 |
| trunk  | surface expressway, divided road, limited number of intersections  | 45 | 45 | 45 | 45 |
| primary  | arterial road  | 30 | 30 | 30 | 30 |
| secondary  | usually separated by a central line on road  | 25 |25 | 25 | 25 |
| tertiary  | links local centers of activity with low to moderate traffic  | 20 | 20 | 20 | 20 |

Note: these tables originally come from Maddie Sheehan's solver code for mapping OSM road types to road speeds/capacities.

In [1]:
import ast
import statistics
import pandas as pd

In [2]:
# define speed defaults for each hwy type and number of lanes, so we can infer when lacking data
speed_defaults = {'tertiary' : {1 : 20, 2 : 20, 3 : 20, 4 : 20, -1 : 20},
                  'tertiary_link' : {1 : 20, 2 : 20, 3 : 20, 4 : 20, -1 : 20},
                  'secondary' : {1 : 25, 2 : 25, 3 : 25, 4 : 25, -1 : 25},
                  'secondary_link' : {1 : 25, 2 : 25, 3 : 25, 4 : 25, -1 : 25},
                  'primary' : {1 : 30, 2 : 30, 3 : 30, 4 : 30, -1 : 30},
                  'primary_link' : {1 : 30, 2 : 30, 3 : 30, 4 : 30, -1 : 30},
                  'trunk' : {1 : 45, 2 : 45, 3 : 45, 4 : 45, -1 : 45},
                  'trunk_link' : {1 : 45, 2 : 45, 3 : 45, 4 : 45, -1 : 45},
                  'motorway' : {1 : 50, 2 : 50, 3 : 65, 4 : 65, -1 : 57.5},
                  'motorway_link' : {1 : 50, 2 : 50, 3 : 65, 4 : 65, -1 : 57.5},
                  'unclassified' : {1 : 20, 2 : 20, 3 : 20, 4 : 20, -1 : 20},
                  'road' : {1 : 30, 2 : 30, 3 : 30, 4 : 30, -1 : 30}}

# define capacity defaults for each hwy type and number of lanes, so we can infer when lacking data
capacity_defaults = {'tertiary' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'tertiary_link' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'secondary' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'secondary_link' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900},
                     'primary' : {1 : 1000, 2 : 1000, 3 : 1000, 4 : 1000, -1 : 1000},
                     'primary_link' : {1 : 1000, 2 : 1000, 3 : 1000, 4 : 1000, -1 : 1000},
                     'trunk' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2000, -1 : 1975},
                     'trunk_link' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2000, -1 : 1975},
                     'motorway' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2200, -1 : 2025},
                     'motorway_link' : {1 : 1900, 2 : 2000, 3 : 2000, 4 : 2200, -1 : 2025},
                     'unclassified' : {1 : 800, 2 : 800, 3 : 800, 4 : 800, -1 : 800},
                     'road' : {1 : 900, 2 : 900, 3 : 900, 4 : 900, -1 : 900}}

# note: -1 is the key for the null value
# note: highway_links are given the same values as their highway types
# note: 'road' is effectively an OSM null highway type
# note: 'unclassified' is a highway type one step below tertiary in the OSM hierarchy

## Load the data

In [3]:
# load the edge data
edges_all = pd.read_csv('data/bay_area_edges.csv', low_memory=False)
len(edges_all)

65716

In [4]:
# retain only useful columns
useful_cols = ['uniqueid', 'u', 'v', 'length', 'maxspeed', 'lanes', 'highway']
ed = edges_all[useful_cols].copy()
ed.count()

uniqueid    65716
u           65716
v           65716
length      65716
maxspeed    17512
lanes       28487
highway     65716
dtype: int64

## Process `highway` type values

In [5]:
# convert string representations of lists to lists
def convert_lists(value):
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        return ast.literal_eval(value) #parse str -> list
    else:
        return value

In [6]:
# convert string representation of multiple highway types to a list
ed['highway'] = ed['highway'].map(convert_lists)

In [7]:
# collapse multiple highway type values into a single value
def collapse_multiple_hwy_values(hwy):
    if isinstance(hwy, list):
        # if we find an item in our defaults dict, use that value
        # otherwise, just use the zeroth item in the list
        for item in hwy:
            if item in speed_defaults.keys():
                return item
        return hwy[0]
    else:
        return hwy

ed['highway'] = ed['highway'].map(collapse_multiple_hwy_values)

In [8]:
ed['highway'].value_counts().sort_index()

motorway           2691
motorway_link      4884
primary            8634
primary_link       1177
road                170
secondary         17297
secondary_link     1048
tertiary          18817
tertiary_link       507
trunk              1977
trunk_link          971
unclassified       7543
Name: highway, dtype: int64

## Process number of `lanes` values

In [9]:
# convert string representation of multiple lane values to a list
ed['lanes'] = ed['lanes'].map(convert_lists)

In [10]:
# collapse multiple lane values into a single value
def collapse_multiple_lane_values(value):
    if isinstance(value, list):
        # return the mean of the values in the list
        numeric_values = [int(x) for x in value]
        return int(statistics.mean(numeric_values))
    else:
        return value

ed['lanes'] = ed['lanes'].map(collapse_multiple_lane_values)

In [11]:
# replace nulls with -1 for dict lookup, and make 4 the max value
ed['lanes'] = ed['lanes'].fillna(value=-1).astype(int)
ed.loc[ed['lanes'] > 4, 'lanes'] = 4

In [12]:
ed['lanes'].value_counts().sort_index()

-1    37229
 1     4048
 2    11641
 3     6117
 4     6681
Name: lanes, dtype: int64

## Process `maxspeed` values

In [13]:
# convert string representation of multiple maxspeed values to a list
ed['maxspeed'] = ed['maxspeed'].map(convert_lists)

In [14]:
# collapse multiple maxspeed values into a single value
def collapse_multiple_maxspeed_values(value):
    if isinstance(value, list):
        try:
            # strip non-numeric " mph" from each value in the list then take the mean
            values = [int(x.replace(' mph', '')) for x in value]
            return statistics.mean(values)
        except:
            # if exception, return null (a few have multiple values like "35 mph;40 mph")
            return None
    else:
        return value

ed['maxspeed'] = ed['maxspeed'].map(collapse_multiple_maxspeed_values)

In [15]:
# for all string maxspeed values, strip non-numeric " mph" from each value
def parse_speed_strings(value):
    if isinstance(value, str):
        value = value.replace(' mph', '')
        # sometimes multiple speeds are semicolon-delimited -- collapse to a single value
        if ';' in value:
            # return the mean of the values if it has that semicolon
            values = [int(x) for x in value.split(';')]
            return statistics.mean(values)
        else:
            return int(value)
    else:
        return value
    
ed['maxspeed'] = ed['maxspeed'].map(parse_speed_strings)

In [16]:
ed['maxspeed'].value_counts().sort_index()

5.000000        1
10.000000      11
15.000000      44
17.500000       4
20.000000      45
22.500000       8
25.000000    3579
26.666667       2
27.500000      44
30.000000    2134
32.500000      80
35.000000    4426
36.250000       4
37.500000      89
38.333333       6
40.000000    1878
41.000000       2
41.250000       2
41.666667       2
42.500000      34
43.333333       2
44.000000       2
45.000000    1756
47.500000      29
50.000000     778
52.500000      23
55.000000     451
56.333333       2
56.666667       4
57.500000       4
60.000000      18
65.000000    1989
67.500000       2
70.000000      44
80.000000       7
Name: maxspeed, dtype: int64

## Calculate free-flow speed for each edge

In [17]:
# infer speed from defaults based on highway type classifier and number of lanes
def infer_speed(row):
    hwy = row['highway']
    lanes = row['lanes']
    return speed_defaults[hwy][lanes]

In [18]:
# extract maxspeed from OSM data when it already exists
known_speeds = ed[pd.notnull(ed['maxspeed'])]['maxspeed']
known_speeds = known_speeds.astype(int)

In [19]:
# infer speed on all other edges that lack maxspeed data
inferred_speeds = ed[pd.isnull(ed['maxspeed'])].apply(infer_speed, axis=1)

# merge known speeds with inferred speeds to get a free-flow speed for each edge
ed['speed_mph'] = known_speeds.append(inferred_speeds, ignore_index=False, verify_integrity=True)

## Calculate capacity for each edge

In [20]:
# infer capacity from defaults based on highway type classifier and number of lanes
def infer_capacity(row):
    hwy = row['highway']
    lanes = row['lanes']
    return capacity_defaults[hwy][lanes]

In [21]:
# infer capacity for each edge using capacity defaults
ed['capacity'] = ed.apply(infer_capacity, axis=1)

In [22]:
ed.head()

,uniqueid,u,v,length,maxspeed,lanes,highway,speed_mph,capacity
0,0,25457926,1696612039,481.148607,NaN,-1,motorway_link,57.5,2025
1,1,25457926,1700584707,633.593245,65.0,3,motorway,65.0,2000
2,2,25457946,260543593,1111.941089,65.0,3,motorway,65.0,2000
3,3,26392420,306525216,92.178443,NaN,-1,tertiary,20.0,900
4,4,26392420,2663869153,28.537692,NaN,-1,unclassified,20.0,800


## Calculate BPR coefficients

In [23]:
# define the parameter values (see documentation at top)
alpha = 0.15
beta = 4
meters_per_mile = 1609.34 #edge length is in meters, but speeds are in miles per hour
seconds_per_hour = 60 * 60 #we want travel times in seconds but speeds are in miles per hour

In [24]:
# calculate a0 as free-flow travel time for each edge, in seconds
ed['length_miles'] = ed['length'] / meters_per_mile
ed['a0'] = ed['length_miles'] / ed['speed_mph']
ed['a0'] = ed['a0'] * seconds_per_hour

In [25]:
# a1, a2, and a3 are not used, they are always zero
ed['a1'] = 0
ed['a2'] = 0
ed['a3'] = 0

In [26]:
# calculate a4 = a0 * alpha / (capacity ^ beta)
ed['a4'] = ed['a0'] * alpha / (ed['capacity'] ** beta)

## Save to disk

In [27]:
# reformat data into format LBNL desires
save_cols = ['uniqueid', 'u', 'v', 'a0', 'a1', 'a2', 'a3', 'a4']
ed_save = ed[save_cols]
ed_save = ed_save.rename(columns={'uniqueid':'LINK', 'u':'A', 'v':'B'})
ed_save.head(10)

,LINK,A,B,a0,a1,a2,a3,a4
0,0,25457926,1696612039,18.718286,0,0,0,1.669772e-13
1,1,25457926,1700584707,21.804788,0,0,0,2.044199e-13
2,2,25457946,260543593,38.266886,0,0,0,3.587521e-13
3,3,26392420,306525216,10.309891,0,0,0,2.357085e-12
4,4,26392420,2663869153,3.191858,0,0,0,1.168893e-12
5,5,26392420,293068494,22.508782,0,0,0,2.219090e-13
6,6,26404828,2876495476,1.671894,0,0,0,3.822346e-13
7,7,26404828,2876495468,21.390974,0,0,0,4.890483e-12
8,8,26404828,2399877571,17.653330,0,0,0,4.035969e-12
9,9,26404847,26409219,12.833325,0,0,0,1.924999e-12


In [28]:
# save as csv file
ed_save.to_csv('data/bay_area_bpr.csv', index=False, encoding='utf-8')